[Source](https://www.kaggle.com/code/bryanfendis/sentiment-analysis-dl)

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./datasets/DfCleaned.csv')

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import LSTM, Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

2023-11-10 13:51:08.405180: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 13:51:08.577791: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 13:51:08.577826: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 13:51:08.579009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 13:51:08.668268: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 13:51:08.669116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Model n-gram3

In [3]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df['ngram3'].values)
x = tokenizer.texts_to_sequences(df['ngram3'].values)
x = pad_sequences(x)

In [4]:
y = pd.get_dummies(df['clarity']).values

In [5]:
from sklearn.model_selection import KFold

In [6]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [7]:
def modelLSTM():
    max_features = 1000
    embed_dim = 32
    lstm_out = 32
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(128, dropout=0.1, recurrent_dropout=0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [10]:
clf = KerasClassifier(build_fn=modelLSTM, epochs=10, batch_size=32, verbose=1)

In [11]:
resultNgram = cross_val_score(clf, x, y, scoring='accuracy', cv=3)
print(resultNgram)
resultNgram.mean()

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[0.4515 0.4495 0.4265]


0.44250000000000006

### Model PoS

In [12]:
df_pos = pd.read_csv('./datasets/DfPos.csv')

In [13]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df_pos['pos'].values)
x = tokenizer.texts_to_sequences(df_pos['pos'].values)
x = pad_sequences(x)

In [14]:
y = pd.get_dummies(df['clarity']).values

In [15]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [16]:
def modelLSTMPos():
    max_features = 1000
    embed_dim = 32
    lstm_out = 32
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(128, dropout=0.1, recurrent_dropout=0.1),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [17]:
clf = KerasClassifier(build_fn=modelLSTMPos, epochs=10, batch_size=32, verbose=1)

In [18]:
from sklearn.model_selection import KFold

In [19]:
resultPos = cross_val_score(clf, x, y, scoring='accuracy', cv=3)
print(resultPos)
resultPos.mean()

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
125/125 [==============================] - 192s 2s/step - loss: 0.6382 - accuracy: 0.3532
Epoch 2/10
125/125 [==============================] - 193s 2s/step - loss: 0.5986 - accuracy: 0.4593
Epoch 3/10
125/125 [==============================] - 199s 2s/step - loss: 0.5668 - accuracy: 0.4995
Epoch 4/10
125/125 [==============================] - 193s 2s/step - loss: 0.5505 - accuracy: 0.5385
Epoch 5/10
125/125 [==============================] - 185s 1s/step - loss: 0.5379 - accuracy: 0.5533
Epoch 6/10
125/125 [==============================] - 182s 1s/step - loss: 0.5230 - accuracy: 0.5813
Epoch 7/10
125/125 [==============================] - 193s 2s/step - loss: 0.5104 - accuracy: 0.5903
Epoch 8/10
125/125 [==============================] - 193s 2s/step - loss: 0.4952 - accuracy: 0.6075
Epoch 9/10
125/125 [==============================] - 194s 2s/step - loss: 0.4905 - accuracy: 0.6143
Epoch 10/10
63/63 [==============================] - 18s 277ms/step


/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
125/125 [==============================] - 215s 2s/step - loss: 0.6372 - accuracy: 0.3605
Epoch 2/10
125/125 [==============================] - 186s 1s/step - loss: 0.5865 - accuracy: 0.4868
Epoch 3/10
125/125 [==============================] - 187s 1s/step - loss: 0.5607 - accuracy: 0.5260
Epoch 4/10
125/125 [==============================] - 176s 1s/step - loss: 0.5447 - accuracy: 0.5493
Epoch 5/10
125/125 [==============================] - 189s 2s/step - loss: 0.5308 - accuracy: 0.5648
Epoch 6/10
125/125 [==============================] - 188s 2s/step - loss: 0.5143 - accuracy: 0.5900
Epoch 7/10
125/125 [==============================] - 190s 2s/step - loss: 0.4934 - accuracy: 0.6205
Epoch 8/10
125/125 [==============================] - 182s 1s/step - loss: 0.4779 - accuracy: 0.6338
Epoch 9/10
125/125 [==============================] - 180s 1s/step - loss: 0.4628 - accuracy: 0.6420
Epoch 10/10
63/63 [==============================] - 18s 273ms/step


/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
125/125 [==============================] - 183s 1s/step - loss: 0.6400 - accuracy: 0.3450
Epoch 2/10
125/125 [==============================] - 180s 1s/step - loss: 0.6042 - accuracy: 0.4457
Epoch 3/10
125/125 [==============================] - 183s 1s/step - loss: 0.5645 - accuracy: 0.5215
Epoch 4/10
125/125 [==============================] - 183s 1s/step - loss: 0.5387 - accuracy: 0.5562
Epoch 5/10
125/125 [==============================] - 191s 2s/step - loss: 0.5176 - accuracy: 0.5947
Epoch 6/10
125/125 [==============================] - 193s 2s/step - loss: 0.4967 - accuracy: 0.6108
Epoch 7/10
125/125 [==============================] - 189s 2s/step - loss: 0.4864 - accuracy: 0.6263
Epoch 8/10
125/125 [==============================] - 194s 2s/step - loss: 0.4727 - accuracy: 0.6455
Epoch 9/10
125/125 [==============================] - 193s 2s/step - loss: 0.4525 - accuracy: 0.6580
Epoch 10/10
63/63 [==============================] - 18s 288ms/step
[0.434  0.4525 0.42  ]


0.4355

In [21]:
x_train,x_test, y_train, y_test = train_test_split(x, y, train_size= 0.8, random_state= 42)

In [ ]:
res = cross_val_score(clf, x_train, y_train, scoring='accuracy', cv=3)


In [ ]:
y_pred = clf.predict(x_test)



In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

con_matrix = confusion_matrix(y_test, y_pred)
print(con_matrix)